<a href="https://colab.research.google.com/github/xxristoskk/stable-audio-sample-generator/blob/main/Stable_Audio_Sample_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Audio Sample Generator

This notebook use's StabilityAI's stable-audio-tools to generate audio samples. You will need to have a HuggingFace account, because a token is required to use the model.

You will be asked to restart the runtime after the following cell is finished running, and you won't need to run it again.

Instrument names are the names of the root folders you want, and the project name is what you want to name the generated audio.

For example, I want to create samples of various guitar playing styles. I can name the instrument Guitar, then name a project metal, punk, jazz, ect. A folder named "Guitar" will be created and a subfolder of the project name.

Create as many instruments and projects as you'd like, and you can zip them together for a download in the last cell. Alternatively, you can download individual files from the file view on the left.

In [1]:
#@markdown #Install Pre-reqs
# Install required libraries
!pip install torch torchaudio einops huggingface_hub stable-audio-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [1]:
import os
import torch
import torchaudio
from einops import rearrange
from stable_audio_tools import get_pretrained_model
from stable_audio_tools.inference.generation import generate_diffusion_cond
from huggingface_hub import login
from google.colab import files

# Function to read text file
# def read_txt(path: str):
#     with open(path, 'r') as file:
#         prompts = [line.strip() for line in file if line.strip()]
#     return prompts

#@markdown #Setup
# Authenticate with Hugging Face
huggingface_token = '' #@param {type: 'string'}
login(huggingface_token)

# Check CUDA availability
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available. Using CPU.")

# Additional diagnostics
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CuDNN version:", torch.backends.cudnn.version())
print("CUDA path:", os.getenv("CUDA_PATH"))
print("CUDA visible devices:", os.getenv("CUDA_VISIBLE_DEVICES"))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Get the model and define sample rate and size
model, model_config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
sample_rate = model_config["sample_rate"]
sample_size = model_config["sample_size"]
model = model.to(device)

# Function to run the model
def run_model(prompt: str, seconds: int, steps: int, output_name: str):
    conditioning = [{
        "prompt": prompt,
        "seconds_start": 0,
        "seconds_total": seconds
    }]
    output = generate_diffusion_cond(
        model,
        steps=steps,
        cfg_scale=cfg_scale,
        conditioning=conditioning,
        sample_size=sample_size,
        sigma_min=0.3,
        sigma_max=500,
        sampler_type="dpmpp-3m-sde",
        device=device
    )
    output = rearrange(output, "b d n -> d (b n)")
    output = output.to(torch.float32).div(torch.max(torch.abs(output))).clamp(-1, 1).mul(32767).to(torch.int16).cpu()
    torchaudio.save(output_name, output, sample_rate)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
CUDA available: True
Using GPU: Tesla T4
PyTorch version: 2.3.0+cu121
CUDA version: 12.1
CuDNN version: 8902
CUDA path: None
CUDA visible devices: None


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


No module named 'flash_attn'
flash_attn not installed, disabling Flash Attention


model.safetensors:  42%|####2     | 377M/892M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


model.safetensors:   0%|          | 0.00/4.85G [00:00<?, ?B/s]

# Settings

In [19]:
# User input for parameters
prompt = 'a bear trying to sing in gegorian chants' #@param {type: 'string'}
steps = 93 #@param {type: 'integer'}
seconds = 30 #@param {type: 'integer'}
cfg_scale = 10 #@param {type: 'integer'}
project_name = 'bear' #@param {type: 'string'}
instrument = 'vox' #@param {type: 'string'}
multiprompt = False #@param {type: 'boolean'}
batch = 3 #@param {type: 'integer'}


In [15]:
#Place your multiple prompts here
prompts = [
    'detroit techno loop',
    'jazz drums at 160bpm',
    'hardcore breakdown drum beat played with chopsticks',
    'the sound of a frustrated bear trying to play progressive rock drums',
]

In [20]:
#@markdown #Run
# Create new directory for outputs
# new_dir = os.path.join(f'{os.getcwd()}/{instrument}', project_name)
new_dir = os.path.join(f'{instrument}', project_name)

os.makedirs(new_dir, exist_ok=True)

# Generate audio samples
if not multiprompt and not batch:
    run_model(prompt, seconds, steps, f'{new_dir}/{project_name}.wav')
elif batch and not multiprompt:
    for x in range(batch):
        run_model(prompt, seconds, steps, f'{new_dir}/{project_name}[{x}].wav')
else:
    for x in range(len(prompts)):
        print(f'Prompt: {prompts[x]}')
        for i in range(batch):
            run_model(prompts[x], seconds, steps, f"{new_dir}/{project_name}[{x}]_{i}.wav")

print('DONE!')


2763077869


  0%|          | 0/93 [00:00<?, ?it/s]

633094682


  0%|          | 0/93 [00:00<?, ?it/s]

3681577299


  0%|          | 0/93 [00:00<?, ?it/s]

DONE!


# Download

In [24]:
# Download the generated files
!zip -r bear.zip {new_dir}
files.download("bear.zip")

updating: vox/bear/ (stored 0%)
updating: vox/bear/bear[0].wav (deflated 41%)
updating: vox/bear/bear[1].wav (deflated 45%)
updating: vox/bear/bear[2].wav (deflated 46%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>